In [1]:
# Step 1

import pandas as pd
import numpy as np
import geopandas as gpd
from haversine import haversine, Unit
import gurobipy as gp
from gurobipy import GRB

# Load and prepare data
nta_table = pd.read_csv("data/model/nta_table.csv")
pantries_table = pd.read_csv("data/model/pantries_table.csv")

# Clean supply gap column
SUPPLY_GAP_COL = "Supply Gap (lbs.)"
nta_table[SUPPLY_GAP_COL] = nta_table[SUPPLY_GAP_COL].apply(
    lambda x: float(str(x).replace(',', ''))
)

print(f"Loaded {len(nta_table)} neighborhoods and {len(pantries_table)} pantries")

# Separate excess and deficit NTAs
EXCESS_NTA = nta_table[nta_table[SUPPLY_GAP_COL] < 0].copy()
DEFICIT_NTA = nta_table[nta_table[SUPPLY_GAP_COL] > 0].copy()

excess_nta_ids = set(EXCESS_NTA['nta2020'].astype(str).str.strip().str.upper())
deficit_nta_ids = set(DEFICIT_NTA['nta2020'].astype(str).str.strip().str.upper())

print(f"\nExcess NTAs: {len(excess_nta_ids)}")
print(f"Deficit NTAs: {len(deficit_nta_ids)}")

deficit_pantries = pantries_table[pantries_table['nta2020'].isin(deficit_nta_ids)]
excess_pantries = pantries_table[pantries_table['nta2020'].isin(excess_nta_ids)]

print(f"\nDeficit pantries: {len(deficit_pantries)}")
print(f"Excess pantries: {len(excess_pantries)}")

Loaded 197 neighborhoods and 515 pantries

Excess NTAs: 55
Deficit NTAs: 142

Deficit pantries: 283
Excess pantries: 231


In [2]:
def parse_point(geom_str):
    """Extract (lat, lon) from POINT string"""
    if pd.isna(geom_str):
        return None
    try:
        # Handle "POINT (lon lat)" format
        coords = str(geom_str).replace('POINT (', '').replace(')', '').split()
        return (float(coords[1]), float(coords[0]))  # (lat, lon)
    except:
        return None

excess_pantries['coords'] = excess_pantries['geometry'].apply(parse_point)
deficit_pantries['coords'] = deficit_pantries['geometry'].apply(parse_point)

# Remove any pantries with missing coordinates
excess_pantries = excess_pantries.dropna(subset=['coords'])
deficit_pantries = deficit_pantries.dropna(subset=['coords'])

print(f"Excess pantries with valid coords: {len(excess_pantries)}")
print(f"Deficit pantries with valid coords: {len(deficit_pantries)}")

excess_pantries['id'] = excess_pantries['id'].astype(str)
deficit_pantries['id'] = deficit_pantries['id'].astype(str)
# Get lists of IDs in order
excess_ids = excess_pantries['id'].tolist()
deficit_ids = deficit_pantries['id'].tolist()

# # Deficit pantry IDs as strings (to match Gurobi var keys)
# deficit_ids = [str(pid) for pid in deficit_pantries["id"]]
# excess_ids  = [str(pid) for pid in excess_pantries["id"]]

# Compute distance matrix
dist_matrix = np.zeros((len(excess_ids), len(deficit_ids)))

for i, excess_id in enumerate(excess_ids):
    excess_coord = excess_pantries[excess_pantries['id'] == excess_id]['coords'].iloc[0]
    for j, deficit_id in enumerate(deficit_ids):
        deficit_coord = deficit_pantries[deficit_pantries['id'] == deficit_id]['coords'].iloc[0]
        dist_matrix[i, j] = haversine(excess_coord, deficit_coord, unit=Unit.MILES)

# Create DataFrame with pantry IDs as index/columns
dist_df_pantry = pd.DataFrame(
    dist_matrix,
    index=excess_ids,
    columns=deficit_ids
)

print(f"\nDistance matrix shape: {dist_df_pantry.shape}")
print(f"Distance range: {dist_matrix.min():.2f} to {dist_matrix.max():.2f} miles")

Excess pantries with valid coords: 231
Deficit pantries with valid coords: 283


/var/folders/dj/dxzvsp311dl66s42c2nyqbk00000gn/T/ipykernel_75828/977242351.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excess_pantries['coords'] = excess_pantries['geometry'].apply(parse_point)
/var/folders/dj/dxzvsp311dl66s42c2nyqbk00000gn/T/ipykernel_75828/977242351.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deficit_pantries['coords'] = deficit_pantries['geometry'].apply(parse_point)



Distance matrix shape: (231, 283)
Distance range: 0.04 to 33.10 miles


In [3]:
# Filter pantries by NTA membership
# Ensure pantries table has clean NTA codes
pantries_table['nta2020_clean'] = pantries_table['nta2020'].astype(str).str.strip().str.upper()

tracts = pd.read_csv("data/model/tracts_table.csv")
pantries = pd.read_csv("data/model/pantries_table_new.csv")
# Ensure geoid matches type
pantries_table["geoid"] = pantries_table["geoid"].astype(str)
tracts["geoid"] = tracts["geoid"].astype(str)

########################### FILTER PANTRIES: ID #, GEO ID, NTA, TOTAL POP ###################
excess_pantries = (
    pantries_table
    # 1. filter to excess NTAs
    .loc[pantries_table['nta2020_clean'].isin(excess_nta_ids),
         ['id', 'geoid', 'nta2020_clean']]
    # 2. add tract population from tracts table
    .merge(
        tracts[['geoid', 'TotalPop']],
        on='geoid',
        how='left'
    )
)
print(excess_pantries)
deficit_pantries = (
    pantries_table
    # 1. filter to excess NTAs
    .loc[pantries_table['nta2020_clean'].isin(deficit_nta_ids),
         ['id', 'geoid', 'nta2020_clean']]
    # 2. add tract population from tracts table
    .merge(
        tracts[['geoid', 'TotalPop']],
        on='geoid',
        how='left'
    )
)
# Ensure 'id' column exists and is string
excess_pantries['id'] = excess_pantries['id'].astype(str)
deficit_pantries['id'] = deficit_pantries['id'].astype(str)

print(f"Excess pantries: {len(excess_pantries)}")
print(f"Deficit pantries: {len(deficit_pantries)}")

# Build supply and demand dictionaries using pantry IDs
# Create NTA lookup dictionary for faster access
nta_lookup = {}
for _, row in nta_table.iterrows():
    nta_code = str(row['nta2020']).strip().upper()
    nta_lookup[nta_code] = {
        'gap': float(row[SUPPLY_GAP_COL]),
        'pantry_count': int(row['pantry_count']) if pd.notna(row['pantry_count']) else 1
    }

# Build supply dictionary (keyed by pantry ID)
supply_per_pantry = {}

for _, row in excess_pantries.iterrows():
    pantry_id = str(row['id'])
    nta_code = row['nta2020_clean']

    if nta_code in nta_lookup:
        gap = nta_lookup[nta_code]['gap']
        n_pantries = max(nta_lookup[nta_code]['pantry_count'], 1)

        supply_amount = abs(gap) / n_pantries if gap < 0 else 0.0
    else:
        supply_amount = 0.0

    # store BOTH NTA and supply
    supply_per_pantry[pantry_id] = {
        "nta": nta_code,
        "supply": supply_amount
    }

print(supply_per_pantry)


# Build demand dictionary (keyed by pantry ID)
demand_per_pantry = {}

for _, row in deficit_pantries.iterrows():
    pantry_id = str(row['id'])
    nta_code = row['nta2020_clean']

    if nta_code in nta_lookup:
        gap = nta_lookup[nta_code]['gap']        # This is positive for deficit NTAs
        n_pantries = max(nta_lookup[nta_code]['pantry_count'], 1)

        demand_amount = gap / n_pantries if gap > 0 else 0.0
    else:
        demand_amount = 0.0

    # Store BOTH the NTA and the amount
    demand_per_pantry[pantry_id] = {
        "nta": nta_code,
        "demand": demand_amount
    }

print(demand_per_pantry)


print(f"\nSupply pantries with data: {len(supply_per_pantry)}")
print(f"Demand pantries with data: {len(demand_per_pantry)}")
total_supply = sum(d["supply"] for d in supply_per_pantry.values())
total_demand = sum(d["demand"] for d in demand_per_pantry.values())

print(f"Total supply (lbs): {total_supply:,.2f}")
print(f"Total demand (lbs): {total_demand:,.2f}")


      id        geoid nta2020_clean  TotalPop
0      4  36081026100        QN0203    7098.0
1      6  36081044601        QN1201    3119.0
2     12  36061011401        MN0802    1276.0
3     16  36081038400        QN1205    2420.0
4     19  36081036300        QN0302    2105.0
..   ...          ...           ...       ...
226  495  36081044400        QN1201    3024.0
227  496  36047116000        BK0502    2139.0
228  506  36047036300        BK1601    4076.0
229  507  36061013000        MN0802    3277.0
230  512  36081051200        QN1303    2925.0

[231 rows x 4 columns]
Excess pantries: 231
Deficit pantries: 283
{'4': {'nta': 'QN0203', 'supply': 181568.003356405}, '6': {'nta': 'QN1201', 'supply': 325767.6515344444}, '12': {'nta': 'MN0802', 'supply': 89079.75160856667}, '16': {'nta': 'QN1205', 'supply': 216955.57853827602}, '19': {'nta': 'QN0302', 'supply': 5821.354547653}, '23': {'nta': 'QN0801', 'supply': 58653.483666203334}, '29': {'nta': 'BK1601', 'supply': 285430.9030031125}, '30': 

In [4]:
# Filter distance matrix
print(f"Supply dict keys: {len(supply_per_pantry)}")
print(f"Demand dict keys: {len(demand_per_pantry)}")
print(f"Distance matrix rows (excess): {dist_df_pantry.shape[0]}")
print(f"Distance matrix columns (deficit): {dist_df_pantry.shape[1]}")
print(f"All supply keys in distance index: {set(supply_per_pantry.keys()) == set(dist_df_pantry.index)}")
print(f"All demand keys in distance columns: {set(demand_per_pantry.keys()) == set(dist_df_pantry.columns)}")
print(f"\nTotal supply available: {sum(d['supply'] for d in supply_per_pantry.values()):,.2f} lbs")
print(f"\nTotal demand needed: {sum(d['demand'] for d in demand_per_pantry.values()):,.2f} lbs")
total_supply = sum(v['supply'] for v in supply_per_pantry.values())
total_demand = sum(v['demand'] for v in demand_per_pantry.values())
print(f"Supply/Demand ratio: {total_supply/total_demand:.2%}")

# OPTIMIZATION MODEL
print("BUILDING OPTIMIZATION MODEL")

######################## CREATE WEIGHT VALUE FOR UNMET DEMAND OBJECTIVE ########################################

# Make sure there are no missing pops; if there are, treat as 0
deficit_pantries["TotalPop"] = deficit_pantries["TotalPop"].fillna(0)

# w_unmet[j] = population of the tract around pantry j
w_unmet = {
    str(row["id"]): float(row["TotalPop"])
    for _, row in deficit_pantries.iterrows()
}

print(w_unmet)

# Build model
model = gp.Model("pantry_rebalance")
model.setParam("OutputFlag", 1)

# Decision variables
x = {(i, j): model.addVar(lb=0.0, name=f"x_{i}_{j}") 
     for i in excess_ids for j in deficit_ids}

u = model.addVars(deficit_ids, name="u", lb=0.0)

model.update()

print(f"\nVariables created:")
print(f"  Flow variables (x): {len(x):,}")
print(f"  Unmet variables (u): {len(u):,}")

# Objective function
transport_cost = gp.quicksum(
    dist_df_pantry.loc[i, j] * x[i, j]
    for i in excess_ids for j in deficit_ids
)

unmet_cost = gp.quicksum(
    w_unmet[j] * u[j] for j in deficit_ids
)

model.setObjective(transport_cost + unmet_cost, GRB.MINIMIZE)

print("\nObjective function set: minimize (transport_cost + penalty * unmet_demand)")

# Constraints
print("\nAdding constraints...")

# 1. Demand satisfaction
for j in deficit_ids:
    model.addConstr(
        gp.quicksum(x[i, j] for i in excess_ids) + u[j] == demand_per_pantry[j]["demand"],
        name=f"demand_{j}"
    )

print(f"  Added {len(deficit_ids)} demand constraints")

# 2. Supply limits
for i in excess_ids:
    model.addConstr(
        gp.quicksum(x[i, j] for j in deficit_ids) <= supply_per_pantry[i]["supply"],
        name=f"supply_{i}"
    )

print(f"  Added {len(excess_ids)} supply constraints")

# Solve
print("\n" + "="*60)
print("SOLVING OPTIMIZATION MODEL")
print("="*60 + "\n")

model.optimize()


Supply dict keys: 231
Demand dict keys: 283
Distance matrix rows (excess): 231
Distance matrix columns (deficit): 283
All supply keys in distance index: True
All demand keys in distance columns: True

Total supply available: 50,589,455.30 lbs

Total demand needed: 96,655,913.95 lbs
Supply/Demand ratio: 52.34%
BUILDING OPTIMIZATION MODEL
{'0': 5874.0, '1': 5650.0, '2': 6446.0, '3': 3999.0, '5': 7195.0, '7': 5069.0, '8': 2513.0, '9': 7962.0, '10': 6890.0, '11': 1683.0, '13': 5319.0, '14': 939.0, '15': 10840.0, '17': 0.0, '18': 3717.0, '20': 2988.0, '21': 4218.0, '22': 7195.0, '24': 0.0, '25': 1486.0, '26': 3797.0, '27': 7279.0, '28': 3782.0, '33': 3125.0, '34': 4131.0, '35': 4048.0, '39': 4702.0, '40': 2577.0, '45': 3411.0, '49': 2436.0, '52': 4790.0, '53': 2685.0, '54': 4987.0, '57': 3797.0, '58': 3058.0, '60': 3302.0, '62': 2560.0, '67': 3973.0, '73': 2111.0, '74': 3278.0, '75': 6947.0, '77': 3860.0, '78': 3506.0, '80': 6156.0, '81': 6416.0, '84': 5136.0, '90': 2906.0, '95': 3819.0, '9

In [5]:
if model.status == GRB.OPTIMAL:
    print("✅ OPTIMAL SOLUTION FOUND")
    
    # Extract flows
    flows = []
    for i in excess_ids:
        for j in deficit_ids:
            val = x[i, j].X
            if val > 1e-6:
                flows.append({
                    "from_pantry": i,
                    "to_pantry": j,
                    "lbs": val,
                    "distance_miles": dist_df_pantry.loc[i, j]
                })
    
    flows_df = pd.DataFrame(flows)
    
    # Extract unmet demanda
    demand = {j: demand_per_pantry[j]['demand'] for j in deficit_ids}
    unmet_demand = {j: u[j].X for j in deficit_ids}
    total_unmet = sum(unmet_demand.values())
    
    # Calculate statistics
    total_transported = flows_df['lbs'].sum() if len(flows_df) > 0 else 0
    total_distance = (flows_df['lbs'] * flows_df['distance_miles']).sum() if len(flows_df) > 0 else 0
    avg_distance = flows_df['distance_miles'].mean() if len(flows_df) > 0 else 0
    
    print(f"\n📊 SOLUTION STATISTICS:")
    print(f"  Total flows created: {len(flows)}")
    print(f"  Total food transported: {total_transported:,.2f} lbs")
    print(f"  Total unmet demand: {total_unmet:,.2f} lbs ({total_unmet/sum(demand.values())*100:.1f}% of total demand)")
    print(f"  Total transport cost: {total_distance:,.2f} lb-miles")
    print(f"  Average transport distance: {avg_distance:.2f} miles")
    print(f"  Objective value: {model.objVal:,.2f}")
    
    # Save results
    if len(flows_df) > 0:
        flows_df.to_csv("data/model/optimal_flows.csv", index=False)
        print(f"\n Results saved to data/model/optimal_flows.csv")
        
        print("\n Sample flows:")
        print(flows_df.head(10).to_string(index=False))
    else:
        print("\n  No flows generated (all demand is unmet)")
    
    # Show pantries with highest unmet demand
    if total_unmet > 0:
        unmet_df = pd.DataFrame([
            {"pantry_id": j, "unmet_lbs": unmet_demand[j], "original_demand": demand[j]}
            for j in deficit_ids if unmet_demand[j] > 1
        ]).sort_values("unmet_lbs", ascending=False)
        
        print(f"\n Top 10 pantries with unmet demand:")
        print(unmet_df.head(10).to_string(index=False))

elif model.status == GRB.INFEASIBLE:
    print("\n MODEL IS INFEASIBLE")
    print("Computing IIS (Irreducible Inconsistent Subsystem)...")
    model.computeIIS()
    model.write("model_iis.ilp")
    print("IIS written to model_iis.ilp")
    
elif model.status == GRB.UNBOUNDED:
    print("\n MODEL IS UNBOUNDED")
    
else:
    print(f"\n Optimization failed with status code: {model.status}")
    print("See Gurobi documentation for status code meanings")

✅ OPTIMAL SOLUTION FOUND

📊 SOLUTION STATISTICS:
  Total flows created: 366
  Total food transported: 50,589,455.30 lbs
  Total unmet demand: 46,066,458.65 lbs (47.7% of total demand)
  Total transport cost: 320,239,874.19 lb-miles
  Average transport distance: 6.20 miles
  Objective value: 139,656,972,748.86

 Results saved to data/model/optimal_flows.csv

 Sample flows:
from_pantry to_pantry           lbs  distance_miles
          4       130 181568.003356        6.531941
          6       416 325767.651534       13.089086
         12       282  89079.751609        1.282941
         16       502 216955.578538        4.974975
         19       127   5821.354548        7.067679
         23       114  41534.240392        9.726085
         23       405  17119.243274        9.949624
         29       282 285430.903003        8.004535
         30       124  58213.454394        6.430468
         30       286  51262.844030        6.794728

 Top 10 pantries with unmet demand:
pantry_id    unm